## Descripción del proyecto

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Te han dado una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

In [8]:
#CODIGO PARA CONEXION A BASE DE DATOS:

# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

1. Revisamos los datos de las tablas:

In [9]:
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [19]:
query = "SELECT * FROM authors"
authors_df = pd.io.sql.read_sql(query, con=engine)
authors_df.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [20]:
query = "SELECT * FROM publishers"
publishers_df = pd.io.sql.read_sql(query, con=engine)
publishers_df.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [21]:
query = "SELECT * FROM ratings"
ratings_df = pd.io.sql.read_sql(query, con=engine)
ratings_df.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [22]:
query = "SELECT * FROM reviews"
reviews_df = pd.io.sql.read_sql(query, con=engine)
reviews_df.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


1. Encuentra el número de libros publicados después del 1 de enero de 2000.

In [32]:
query = """
SELECT COUNT(*) AS number_books
FROM books
WHERE publication_date > '2000-01-01'
"""
books_df = pd.io.sql.read_sql(query, con=engine)
print(books_df)

   number_books
0           819


El número de libros publicados después del 1 de enero de 2000 es de 819.

2. Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [34]:
query = """
SELECT 
    rvw.book_id, 
    COUNT(rvw.review_id) AS num_of_reviews, 
    AVG(rt.rating) AS avg_rating
FROM reviews rvw
JOIN ratings rt ON rvw.book_id = rt.book_id
GROUP BY rvw.book_id
"""
reviews_with_ratings_df = pd.io.sql.read_sql(query, con=engine)
print(reviews_with_ratings_df)

     book_id  num_of_reviews  avg_rating
0        652               4    4.500000
1        273               4    4.500000
2         51              60    4.250000
3        951               4    4.000000
4        839              28    4.285714
..       ...             ...         ...
989       64              52    4.230769
990       55               4    5.000000
991      148              21    3.428571
992      790               4    3.500000
993      828               4    3.000000

[994 rows x 3 columns]


La columna 'avg_rating' muestra la calificación promedio para cada libro y la columna 'num_of_reviews' muestra el número de reseñas para cada libro.

3. Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis).

In [36]:
query = """
SELECT
    p.publisher,
    COUNT(b.book_id) AS num_of_books
FROM
    books b
JOIN
    publishers p ON b.publisher_id = p.publisher_id
WHERE
    b.num_pages > 50
GROUP BY
    p.publisher
ORDER BY
    num_of_books DESC
LIMIT 1
"""
books_with_publishers_df = pd.io.sql.read_sql(query, con=engine)
print(books_with_publishers_df)

       publisher  num_of_books
0  Penguin Books            42


La editorial Penguin Books es la que ha publicado el mayor número de libros con más de 50 páginas, con un total de 42 libros.

4. Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [40]:
query = """
SELECT
    a.author,
    AVG(rt.rating) AS avg_rating,
    COUNT(rt.rating) AS number_of_ratings
FROM
    authors a
JOIN
    books b ON a.author_id = b.author_id
JOIN
    ratings rt ON b.book_id = rt.book_id
GROUP BY
    a.author
HAVING
    COUNT(rt.rating) >= 50
ORDER BY 
    avg_rating DESC
LIMIT 1
"""
author_with_books_with_ratings_df = pd.io.sql.read_sql(query, con=engine)
print(author_with_books_with_ratings_df)

           author  avg_rating  number_of_ratings
0  Diana Gabaldon         4.3                 50


El autor que tiene la más alta calificación promedio del libro con al menos 50 calificaciones es Diana Gabaldon.

5. Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [44]:
query = """
WITH user_reviews AS (
    SELECT
        rt.username,
        COUNT(rvw.review_id) AS num_reviews
    FROM
        reviews rvw
    JOIN
        ratings rt ON rvw.book_id = rt.book_id
    GROUP BY
        rt.username
    HAVING
        COUNT(rt.book_id) > 50
)
SELECT
    AVG(num_reviews) AS avg_reviews_per_user
FROM
    user_reviews
"""
reviews_with_ratings_df2 = pd.io.sql.read_sql(query, con=engine)
print(reviews_with_ratings_df2)

   avg_reviews_per_user
0             163.54375


El número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros es de 163.54